In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

In [3]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [4]:
%run load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
x_data = obtain_SWNUNetwork_input(
    method = "gaussian_random_projection",
    dimension= 30,
    df= df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings,
    k=5,
    features='time_encoding',
    standardise_method=None,
    add_time_in_path=False,
)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 415])

## SWNU Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
add_time_in_path = True

In [8]:
num_epochs = 100
embedding_dim = 384
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[64,64],[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
bidirectional = True
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

In [9]:
# #dimensionality reduction
# embedding_dim = sbert_embeddings.shape[1]
# dim_reduce_method = ["umap"] #["gaussian_random_projection", "umap"]
# dimensions = [20]#[50,30,15]
# #time features
# features = "time_encoding"
# standardise_method = "standardise"
# add_time_in_path = False
# #SWNU block
# augmentation_tp = "Conv1d"
# hidden_dim_aug = None
# comb_m = "concatenation"
# log_sig = True
# conv_output_channels = [10] #[20, 10, 5]
# log_signature_dimensions_and_sig_depths = [(8, 3)]#[(30, 2), (10, 3), (6, 4)]
# bidirectional = False
# #ffn
# hidden_dim_sizes = [64]#[32,64]
# dropout_rates = [0.2]#[0.5, 0.2, 0.1]
# #overall training
# num_epochs = 100
# batch=64
# patience = 4
# learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
# seeds = [0, 1, 12, 123, 1234]
# loss = "focal"
# gamma = 2
# validation_metric = "f1"
# split_indices = (df_rumours[df_rumours['set']=='train'].index,
#                  df_rumours[df_rumours['set']=='dev'].index,
#                  df_rumours[df_rumours['set']=='test'].index)

In [10]:
size=35

## UMAP

In [ ]:
swnu_network_umap, best_swnu_network_umap, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [ ]:
swnu_network_umap.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

In [ ]:
best_swnu_network_umap

In [ ]:
best_swnu_network_umap["f1"].mean()

In [ ]:
best_swnu_network_umap["precision"].mean()

In [ ]:
best_swnu_network_umap["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap["recall_scores"]).mean(axis=0)

## Unidirectional LSTM

In [ ]:
swnu_network_umap_uni, best_swnu_network_umap_uni, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_uni.csv",
    verbose=False
)

In [ ]:
swnu_network_umap_uni.groupby(["dimensions",
                               "swnu_hidden_dim",
                               "ffn_hidden_dim",
                               "dropout_rate",
                               "learning_rate"]).mean()

In [ ]:
best_swnu_network_umap_uni

In [ ]:
best_swnu_network_umap_uni["f1"].mean()

In [ ]:
best_swnu_network_umap_uni["precision"].mean()

In [ ]:
best_swnu_network_umap_uni["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_uni["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_uni["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_uni["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
swnu_network_grp, best_swnu_network_grp, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}.csv",
    verbose=False
)

In [ ]:
swnu_network_grp.groupby(["dimensions",
                           "swnu_hidden_dim",
                           "ffn_hidden_dim",
                           "dropout_rate",
                           "learning_rate"]).mean()

In [ ]:
best_swnu_network_grp

In [ ]:
best_swnu_network_grp["f1"].mean()

In [ ]:
best_swnu_network_grp["precision"].mean()

In [ ]:
best_swnu_network_grp["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp["recall_scores"]).mean(axis=0)

## Unidirectional LSTM

In [ ]:
swnu_network_grp_uni, best_swnu_network_grp_uni, _, __ = swnu_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features, 
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    split_ids=None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices=split_indices,
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_uni.csv",
    verbose=False
)

In [ ]:
swnu_network_grp_uni.groupby(["dimensions",
                              "swnu_hidden_dim",
                              "ffn_hidden_dim",
                              "dropout_rate",
                              "learning_rate"]).mean()

In [ ]:
best_swnu_network_grp_uni

In [ ]:
best_swnu_network_grp_uni["f1"].mean()

In [ ]:
best_swnu_network_grp_uni["precision"].mean()

In [ ]:
best_swnu_network_grp_uni["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_uni["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_uni["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_uni["recall_scores"]).mean(axis=0)